##### 问题：
评估多线程应用的性能。

在本节中，我们将验证GIL的影响，评估多线程应用的性能。前文已经介绍过，GIL是CPython解释器引入的锁，GIL在解释器层面阻止了真正的并行运行。解释器在执行任何线程之前，必须等待当前正在运行的线程释放GIL。事实上，解释器会强迫想要运行的线程必须拿到GIL才能访问解释器的任何资源，例如栈或Python对象等。这也正是GIL的目的——阻止不同的线程并发访问Python对象。这样GIL可以保护解释器的内存，让垃圾回收工作正常。但事实上，这却造成了程序员无法通过并行执行多线程来提高程序的性能。如果我们去掉CPython的GIL，就可以让多线程真正并行执行。GIL并没有影响多处理器并行的线程，只是限制了一个解释器只能有一个线程在运行。

下面的代码是用来评估多线程应用性能的简单工具。下面的每一个测试都循环调用函数100次，重复执行多次，取速度最快的一次。在 for 循环中，我们调用 non_threaded 和 threaded 函数。同时，我们会不断增加调用次数和线程数来重复执行这个测试。我们会尝试使用1，2，3，4和8线程数来调用线程。在非线程的测试中，我们顺序调用函数与对应线程数一样多的次数。为了保持简单，度量的指标使用Python的内建模块timer。

代码如下：

In [1]:
from threading import Thread

class threads_object(Thread):
    def run(self):
        function_to_run()

class nothreads_object(object):
    def run(self):
        function_to_run()

def non_threaded(num_iter):
    funcs = []
    for i in range(int(num_iter)):
        funcs.append(nothreads_object())
    for i in funcs:
        i.run()

def threaded(num_threads):
    funcs = []
    for i in range(int(num_threads)):
        funcs.append(threads_object())
    for i in funcs:
        i.start()
    for i in funcs:
        i.join()

def function_to_run():
    pass

def show_results(func_name, results):
    print("%-23s %4.6f seconds" % (func_name, results))

if __name__ == "__main__":
    import sys
    from timeit import Timer
    repeat = 100
    number = 1
    num_threads = [1, 2, 4, 8]
    print('Starting tests')
    for i in num_threads:
        t = Timer("non_threaded(%s)" % i, "from __main__ import non_threaded")
        best_result = min(t.repeat(repeat=repeat, number=number))
        show_results("non_threaded (%s iters)" % i, best_result)
        t = Timer("threaded(%s)" % i, "from __main__ import threaded")
        best_result = min(t.repeat(repeat=repeat, number=number))
        show_results("threaded (%s threads)" % i, best_result)
        print('Iterations complete')

Starting tests
non_threaded (1 iters)  0.000001 seconds
threaded (1 threads)    0.000124 seconds
Iterations complete
non_threaded (2 iters)  0.000001 seconds
threaded (2 threads)    0.000252 seconds
Iterations complete
non_threaded (4 iters)  0.000002 seconds
threaded (4 threads)    0.000540 seconds
Iterations complete
non_threaded (8 iters)  0.000003 seconds
threaded (8 threads)    0.001139 seconds
Iterations complete


我们一共进行了四次测试，每一次都会使用不同的function进行测试，只要改变 function_to_run() 就可以了。

你应该记住，增加线程并不会提高应用启动的时间，但是可以支持并发。例如，一次性创建一个线程池，并重用worker会很有用。这可以让我们切分一个大的数据集，用同样的函数处理不同的部分（生产者消费者模型）。上面这些测试并不是并发应用的模型，只是尽量简单的测试。那么GIL会成为试图发挥多线程应用潜能的纯Python开发的瓶颈吗？是的。线程是编程语言的架构，CPython解释器是线程和操作系统的桥梁。这就是为什么Jython，IronPython没有GIL的原因（Pypy也没有），因为它不是必要的。

